In [5]:
import torch
from openicl import DatasetReader, PromptTemplate, BM25Retriever, CoTInferencer, TopkRetriever, BaseRetriever, RandomRetriever


In [3]:
# Download RoBERTa already finetuned for MNLI
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

with torch.no_grad():
    # Encode a pair of sentences and make a prediction
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 0  # contradiction

    # Encode another pair of sentences
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 2  # entailment

Using cache found in /home/nlonyuk/.cache/torch/hub/pytorch_fairseq_main
2023-06-05 14:42:13 | INFO | fairseq.file_utils | http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz not found in cache, downloading to /tmp/tmp4bthadm0


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


100%|██████████| 751652118/751652118 [02:46<00:00, 4519472.68B/s] 
2023-06-05 14:44:59 | INFO | fairseq.file_utils | copying /tmp/tmp4bthadm0 to cache at /home/nlonyuk/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
2023-06-05 14:45:00 | INFO | fairseq.file_utils | creating metadata file for /home/nlonyuk/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
2023-06-05 14:45:00 | INFO | fairseq.file_utils | removing temp file /tmp/tmp4bthadm0
2023-06-05 14:45:00 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz from cache at /home/nlonyuk/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
2023-06-

In [9]:
data = DatasetReader('gsm8k', name='main',
						input_columns=['question'], output_column='answer', ds_size=5)

prompt = "I bought 5 apples"

train_prompts = [
    "I bought 5 pears",
    "I kicked a ball",
    "20 degrees is hot"
]

predictions = list()
for idx, tp in enumerate(train_prompts):
    print(idx)
    tokens = roberta.encode(prompt, tp)
    prediction = roberta.predict('mnli', tokens).argmax().item()
    predictions.append((tp, prediction))
predictions = sorted(predictions, key=lambda x: x[1])
predictions.reverse()
print(f'prompt to check with:\n {prompt}\n\n\n')
print('results:')
print("\n".join(str(p) for p in predictions))


  0%|          | 0/2 [00:00<?, ?it/s]

0
1
2
prompt to check with:
 I bought 5 apples



('20 degrees is hot', 1)
('I kicked a ball', 0)
('I bought 5 pears', 0)
